In [1]:
# =====================================================================
# Processo de Automatización NOKIA 5G
# =====================================================================

import os, glob, io, re
from pathlib import Path
import pandas as pd
from IPython.lib.deepreload import load_next
from openpyxl import load_workbook
from openpyxl.styles import Alignment, Font, Border, PatternFill
from openpyxl.utils import get_column_letter
import numpy as np
from datetime import date
import unicodedata
from charset_normalizer import from_path


# Ruta base (ajústala si cambia)
#BASE_DIR = Path(r"C:\Users\EAlor\OneDrive - ACS Solutions\Documents\AT&T\LST Cell Ran\Nokia New\Nokia Noviembre")
BASE_DIR = Path(r"C:\Users\SCaracoza\Documents\AT&T\LST Cell Ran\Nokia\XML_Output\Febrero")

HEADERS = ["AT&T_Site_Name", "Site ID", "VERSION", "DISTNAME", "MOID", "angle", "name", "actDl256Qam", "administrativeState", "availabilityStatus", "cellBarred", "cellName", "freqBandIndicatorNR", "lcrId", "nrCellIdentity", "nrCellType", "operationalState", "pMax", "physCellId", "arfcnSsbPbch", "chBwDl", "chBwUl", "nrarfcnDl", "nrarfcnUl", "enbPlmn_mcc_mnc_mncLength", "ltePhyCellId", "ssbPosition", "configuredEpsTac", "nrPlmnDNList", "LAT", "LON"]


# Lista de encabezados, en el orden requerido
HEADER_NRCELL = ["VERSION", "DISTNAME", "MOID", "angle", "name", "actDl256Qam", "administrativeState", "availabilityStatus", "cellBarred", "cellName", "freqBandIndicatorNR", "lcrId", "nrCellIdentity", "nrCellType", "operationalState", "pMax", "physCellId", "arfcnSsbPbch"]

HEADER_MRBTS = ["FILENAME", "DATETIME", "VERSION", "DISTNAME", "MOID", "name", "altitude", "btsName", "latitude", "longitude", "blockingState"]

HEADER_NRCELL_FDD = ["DISTNAME", "chBwDl", "chBwUl", "nrarfcnDl", "nrarfcnUl"]

HEADER_NRDSSLTE = ["DISTNAME", "enbPlmn_mcc_mnc_mncLength", "ltePhyCellId", "ssbPosition"]

HEADER_NRPLMNSET_NSA = ["DISTNAME", "configuredEpsTac", "nrPlmnDNList"]


In [2]:
def read_csv_files(filename: str, header, encoder):
    filepath = str(BASE_DIR / f"{filename}")
    # Leer sólo las columnas necesarias del csv
    df = pd.read_csv(filepath, usecols = header, encoding = encoder, dtype=str)[header]

#    print(nrcell_df.head())

    return df


In [3]:

nrcell_df = read_csv_files("NRCELL.csv", HEADER_NRCELL, "utf-8")
mrbts_df = read_csv_files("MRBTS.csv", HEADER_MRBTS, "latin-1")
nrcell_fdd_df = read_csv_files("NRCELL_FDD.csv", HEADER_NRCELL_FDD, "utf-8")
nrdsslte_df = read_csv_files("NRDSSLTE.csv",HEADER_NRDSSLTE, "utf-8")
nrplmnset_df = read_csv_files("NRPLMNSET_NSA.csv",HEADER_NRPLMNSET_NSA, "utf-8")


print("Shape NRCELL original:", nrcell_df.shape, "\nShape MRBTS original:", mrbts_df.shape, "\nShape NRCELL_FDD original:", nrcell_fdd_df.shape, "\nShape NRDSSLTE original:", nrdsslte_df.shape, "\nShape NRPLMNSET_NSA original:", nrplmnset_df.shape)
# print(NRCELL_fdd_df.head(5).to_string(index=False))

try:
    display(nrcell_df.head(5))
    display(mrbts_df.head(5))
    display(nrcell_fdd_df.head(5))
    display(nrdsslte_df.head(5))
    display(nrplmnset_df.head(5))
except NameError:
    # Por si no estás en notebook
    print(nrcell_df.head(5).to_string(index=False))
    print(mrbts_df.head(5).to_string(index=False))
    print(nrcell_fdd_df.head(5).to_string(index=False))
    print(nrdsslte_df.head(5).to_string(index=False))
    print(nrplmnset_df.head(5).to_string(index=False))



Shape NRCELL original: (2159, 18) 
Shape MRBTS original: (1539, 11) 
Shape NRCELL_FDD original: (2159, 5) 
Shape NRDSSLTE original: (2159, 4) 
Shape NRPLMNSET_NSA original: (2159, 3)


,VERSION,DISTNAME,MOID,angle,name,actDl256Qam,administrativeState,availabilityStatus,cellBarred,cellName,freqBandIndicatorNR,lcrId,nrCellIdentity,nrCellType,operationalState,pMax,physCellId,arfcnSsbPbch
0,NRBTSCL24R1_2322_120,PLMN-PLMN/MRBTS-110705/NRBTS-110705/NRCELL-11,106423692,2.0,NMRGUASAL0477_1_T,true,Unlocked,NaN,notBarred,NMRGUASAL0477_1_T,7,11,1813790731,4DL4UL,enabled,43.0,60,531850
1,NRBTSCL24R1_2322_120,PLMN-PLMN/MRBTS-110705/NRBTS-110705/NRCELL-12,106423694,4.0,NMRGUASAL0477_2_T,true,Unlocked,NaN,notBarred,NMRGUASAL0477_2_T,7,12,1813790732,4DL4UL,enabled,43.0,61,531850
2,NRBTSCL24R1_2322_120,PLMN-PLMN/MRBTS-110705/NRBTS-110705/NRCELL-13,106423695,5.0,NMRGUASAL0477_3_T,true,Unlocked,NaN,notBarred,NMRGUASAL0477_3_T,7,13,1813790733,4DL4UL,enabled,43.0,62,531850
3,NRBTSCL24R1_2322_120,PLMN-PLMN/MRBTS-110724/NRBTS-110724/NRCELL-11,111941765,3.0,NMRGUACEL1699_1_T,true,Unlocked,NaN,notBarred,NMRGUACEL1699_1_T,7,11,1814102027,4DL4UL,enabled,43.0,113,531850
4,NRBTSCL24R1_2322_120,PLMN-PLMN/MRBTS-110724/NRBTS-110724/NRCELL-12,111941763,3.0,NMRGUACEL1699_2_T,true,Unlocked,NaN,notBarred,NMRGUACEL1699_2_T,7,12,1814102028,4DL4UL,enabled,43.0,111,531850


,FILENAME,DATETIME,VERSION,DISTNAME,MOID,name,altitude,btsName,latitude,longitude,blockingState
0,All_Nokia.xml,2026-02-11T11:08:23.000-06:00,SBTS22R2_2112_100,PLMN-PLMN/MRBTS-10001,9678559,NdB_RR_LAB_MER,2261,ToyCell_MERIDA,"+19Â°32'39.627""","-99Â°12'12.098""",NaN
1,All_Nokia.xml,2026-02-11T11:08:23.000-06:00,SBTS22R2_2112_100,PLMN-PLMN/MRBTS-10002,13335834,ToyCell-2,NaN,T-Veracruz,NaN,NaN,NaN
2,All_Nokia.xml,2026-02-11T11:08:23.000-06:00,SBTS24R1_2322_100,PLMN-PLMN/MRBTS-10004,64395764,10004-AGUAGU0025-ESTRELLA,1890,10004-AGUAGU0025-ESTRELLA,"+21Â°54'15.712""","-102Â°17'06.002""",Unblocked
3,All_Nokia.xml,2026-02-11T11:08:23.000-06:00,SBTS24R1_2322_100,PLMN-PLMN/MRBTS-10006,28730021,10006-AGUAGU0008-CAMPESTRE,1870,10006-AGUAGU0008-CAMPESTRE,"+21Â°55'34.323""","-102Â°19'04.979""",Unblocked
4,All_Nokia.xml,2026-02-11T11:08:23.000-06:00,SBTS24R1_2322_100,PLMN-PLMN/MRBTS-10007,57268286,10007-AGUAGU0048-VILLA TERESA,NaN,10007-AGUAGU0048-VILLA TERESA,NaN,NaN,Unblocked


,DISTNAME,chBwDl,chBwUl,nrarfcnDl,nrarfcnUl
0,PLMN-PLMN/MRBTS-110705/NRBTS-110705/NRCELL-11/...,20MHz,20MHz,532000,508000
1,PLMN-PLMN/MRBTS-110705/NRBTS-110705/NRCELL-12/...,20MHz,20MHz,532000,508000
2,PLMN-PLMN/MRBTS-110705/NRBTS-110705/NRCELL-13/...,20MHz,20MHz,532000,508000
3,PLMN-PLMN/MRBTS-110724/NRBTS-110724/NRCELL-11/...,20MHz,20MHz,532000,508000
4,PLMN-PLMN/MRBTS-110724/NRBTS-110724/NRCELL-12/...,20MHz,20MHz,532000,508000


,DISTNAME,enbPlmn_mcc_mnc_mncLength,ltePhyCellId,ssbPosition
0,PLMN-PLMN/MRBTS-110705/NRBTS-110705/NRCELL-11/...,334&50&3,60,pos1
1,PLMN-PLMN/MRBTS-110705/NRBTS-110705/NRCELL-12/...,334&50&3,61,pos1
2,PLMN-PLMN/MRBTS-110705/NRBTS-110705/NRCELL-13/...,334&50&3,62,pos1
3,PLMN-PLMN/MRBTS-110724/NRBTS-110724/NRCELL-11/...,334&50&3,113,pos1
4,PLMN-PLMN/MRBTS-110724/NRBTS-110724/NRCELL-12/...,334&50&3,111,pos1


,DISTNAME,configuredEpsTac,nrPlmnDNList
0,PLMN-PLMN/MRBTS-110705/NRBTS-110705/NRCELL-11/...,6170,MRBTS-110705/NRBTS-110705/NRPLMN-1;MRBTS-11070...
1,PLMN-PLMN/MRBTS-110705/NRBTS-110705/NRCELL-12/...,6170,MRBTS-110705/NRBTS-110705/NRPLMN-1;MRBTS-11070...
2,PLMN-PLMN/MRBTS-110705/NRBTS-110705/NRCELL-13/...,6170,MRBTS-110705/NRBTS-110705/NRPLMN-1;MRBTS-11070...
3,PLMN-PLMN/MRBTS-110724/NRBTS-110724/NRCELL-11/...,6115,MRBTS-110724/NRBTS-110724/NRPLMN-1;MRBTS-11072...
4,PLMN-PLMN/MRBTS-110724/NRBTS-110724/NRCELL-12/...,6115,MRBTS-110724/NRBTS-110724/NRPLMN-1;MRBTS-11072...


In [4]:
# =====================================================================
# 1) Insertar columnas AT&TSite_Name y Site ID
# =====================================================================

# Copia para no tocar los originales
nrcell_df_mod = nrcell_df.copy()


print("Shape original:", nrcell_df_mod.shape)

# --- Crear columna 'AT&T_Site_Name' a partir de mrbts_df ---
# 1. Crear un mapeo DISTNAME -> name
mapa = dict(zip(mrbts_df['DISTNAME'], mrbts_df['name']))

# 2. Generar la clave intermedia a partir de DISTNAME
substr_distname = nrcell_df_mod['DISTNAME'].str.split('/', n=2).str[:2].str.join('/')

# 3. Mapear con el diccionario
tmp_name = substr_distname.map(mapa)

# 4. Aplicar la lógica condicional:
#    - Si contiene '-', usar la segunda parte (x.split('-', 2)[1])
#    - Si no, dejar el valor original
#    - Si está vacío, dejar NaN
nrcell_df_mod.insert(
    0,
    'AT&T_Site_Name',
    tmp_name.apply(
        lambda x: x.split('-', 2)[1].strip() if isinstance(x, str) and '-' in x else x
    )
)

# 5. Rellenar los NaN con el valor original de mrbts_df['name']
nrcell_df_mod['AT&T_Site_Name'].fillna(substr_distname.map(mapa), inplace=True)

# 6. Contar y reportar cuántos valores quedaron vacíos
nan_count = nrcell_df_mod['AT&T_Site_Name'].isna().sum()
if nan_count > 0:
    print(f"⚠️ {nan_count} registros sin 'AT&T_Site_Name' encontrados.")
else:
    print("✅ Todos los registros tienen 'AT&T_Site_Name'.")

# Insertar la nueva columna Site ID extrayendo el texto deseado de DISTNAME
nrcell_df_mod.insert(1, 'Site ID', nrcell_df_mod['DISTNAME'].str.split('-', n=3).str[3].str.split('/',n=2).str[0] )

print("Shape nuevo:", nrcell_df_mod.shape)

# Vista de verificación (muestra solo unas filas)
pd.set_option("display.max_columns", None)  # opcional
try:
    display(nrcell_df_mod.head(5))
except NameError:
    # Por si no estás en notebook
    print(nrcell_df_mod.head(5).to_string(index=False))


Shape original: (2159, 18)
⚠️ 3 registros sin 'AT&T_Site_Name' encontrados.
Shape nuevo: (2159, 20)


C:\Users\SCaracoza\AppData\Local\Temp\ipykernel_28776\1614154696.py:34: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  nrcell_df_mod['AT&T_Site_Name'].fillna(substr_distname.map(mapa), inplace=True)


,AT&T_Site_Name,Site ID,VERSION,DISTNAME,MOID,angle,name,actDl256Qam,administrativeState,availabilityStatus,cellBarred,cellName,freqBandIndicatorNR,lcrId,nrCellIdentity,nrCellType,operationalState,pMax,physCellId,arfcnSsbPbch
0,GUASAL0477,110705,NRBTSCL24R1_2322_120,PLMN-PLMN/MRBTS-110705/NRBTS-110705/NRCELL-11,106423692,2.0,NMRGUASAL0477_1_T,true,Unlocked,NaN,notBarred,NMRGUASAL0477_1_T,7,11,1813790731,4DL4UL,enabled,43.0,60,531850
1,GUASAL0477,110705,NRBTSCL24R1_2322_120,PLMN-PLMN/MRBTS-110705/NRBTS-110705/NRCELL-12,106423694,4.0,NMRGUASAL0477_2_T,true,Unlocked,NaN,notBarred,NMRGUASAL0477_2_T,7,12,1813790732,4DL4UL,enabled,43.0,61,531850
2,GUASAL0477,110705,NRBTSCL24R1_2322_120,PLMN-PLMN/MRBTS-110705/NRBTS-110705/NRCELL-13,106423695,5.0,NMRGUASAL0477_3_T,true,Unlocked,NaN,notBarred,NMRGUASAL0477_3_T,7,13,1813790733,4DL4UL,enabled,43.0,62,531850
3,GUACEL1699,110724,NRBTSCL24R1_2322_120,PLMN-PLMN/MRBTS-110724/NRBTS-110724/NRCELL-11,111941765,3.0,NMRGUACEL1699_1_T,true,Unlocked,NaN,notBarred,NMRGUACEL1699_1_T,7,11,1814102027,4DL4UL,enabled,43.0,113,531850
4,GUACEL1699,110724,NRBTSCL24R1_2322_120,PLMN-PLMN/MRBTS-110724/NRBTS-110724/NRCELL-12,111941763,3.0,NMRGUACEL1699_2_T,true,Unlocked,NaN,notBarred,NMRGUACEL1699_2_T,7,12,1814102028,4DL4UL,enabled,43.0,111,531850


In [5]:
# =====================================================================
# 2) Información de NRCELL_FDD
# =====================================================================

nrcell_df_merged = nrcell_df_mod.copy()

print("Shape original:", nrcell_df_merged.shape)

# Remueve la ultima parte de DISTNAME en NRCELL_FDD para que haga match con DISTNAME de NRCELL
nrcell_fdd_df['DISTNAME'] = nrcell_fdd_df['DISTNAME'].str.rsplit('/', n=1).str[0]

columnas_a_insertar = ["chBwDl", "chBwUl", "nrarfcnDl", "nrarfcnUl"]

# limpia string de DISTNAME
nrcell_df_merged["DISTNAME"] = nrcell_df_merged["DISTNAME"].astype(str).str.strip().str.upper()
nrcell_fdd_df["DISTNAME"] = nrcell_fdd_df["DISTNAME"].astype(str).str.strip().str.upper()

coincidencias = set(nrcell_df_merged["DISTNAME"]) & set(nrcell_fdd_df["DISTNAME"])
print(len(coincidencias))

nrcell_df_merged = nrcell_df_merged.merge(
    nrcell_fdd_df[["DISTNAME"] + columnas_a_insertar],
    on = "DISTNAME",
    how = "left")

print("Shape nuevo:", nrcell_df_merged.shape)

# Vista de verificación (muestra solo unas filas)
pd.set_option("display.max_columns", None)  # opcional
try:
    display(nrcell_df_merged.head(5))
except NameError:
    # Por si no estás en notebook
    print(nrcell_df_merged.head(5).to_string(index=False))


Shape original: (2159, 20)
2159
Shape nuevo: (2159, 24)


,AT&T_Site_Name,Site ID,VERSION,DISTNAME,MOID,angle,name,actDl256Qam,administrativeState,availabilityStatus,cellBarred,cellName,freqBandIndicatorNR,lcrId,nrCellIdentity,nrCellType,operationalState,pMax,physCellId,arfcnSsbPbch,chBwDl,chBwUl,nrarfcnDl,nrarfcnUl
0,GUASAL0477,110705,NRBTSCL24R1_2322_120,PLMN-PLMN/MRBTS-110705/NRBTS-110705/NRCELL-11,106423692,2.0,NMRGUASAL0477_1_T,true,Unlocked,NaN,notBarred,NMRGUASAL0477_1_T,7,11,1813790731,4DL4UL,enabled,43.0,60,531850,20MHz,20MHz,532000,508000
1,GUASAL0477,110705,NRBTSCL24R1_2322_120,PLMN-PLMN/MRBTS-110705/NRBTS-110705/NRCELL-12,106423694,4.0,NMRGUASAL0477_2_T,true,Unlocked,NaN,notBarred,NMRGUASAL0477_2_T,7,12,1813790732,4DL4UL,enabled,43.0,61,531850,20MHz,20MHz,532000,508000
2,GUASAL0477,110705,NRBTSCL24R1_2322_120,PLMN-PLMN/MRBTS-110705/NRBTS-110705/NRCELL-13,106423695,5.0,NMRGUASAL0477_3_T,true,Unlocked,NaN,notBarred,NMRGUASAL0477_3_T,7,13,1813790733,4DL4UL,enabled,43.0,62,531850,20MHz,20MHz,532000,508000
3,GUACEL1699,110724,NRBTSCL24R1_2322_120,PLMN-PLMN/MRBTS-110724/NRBTS-110724/NRCELL-11,111941765,3.0,NMRGUACEL1699_1_T,true,Unlocked,NaN,notBarred,NMRGUACEL1699_1_T,7,11,1814102027,4DL4UL,enabled,43.0,113,531850,20MHz,20MHz,532000,508000
4,GUACEL1699,110724,NRBTSCL24R1_2322_120,PLMN-PLMN/MRBTS-110724/NRBTS-110724/NRCELL-12,111941763,3.0,NMRGUACEL1699_2_T,true,Unlocked,NaN,notBarred,NMRGUACEL1699_2_T,7,12,1814102028,4DL4UL,enabled,43.0,111,531850,20MHz,20MHz,532000,508000


In [6]:
# =====================================================================
# 3) Información de NRDSSLTE
# =====================================================================

nrcell_df_mergelte = nrcell_df_merged.copy()

print("Shape original:", nrcell_df_mergelte.shape)

# Remueve la ultima parate de DISTNAME en NRDSSLTE para que haga match con DISTNAME de NRCELL
nrdsslte_df['DISTNAME'] = nrdsslte_df['DISTNAME'].str.rsplit('/', n=1).str[0]

columnas_a_insertar = ["enbPlmn_mcc_mnc_mncLength", "ltePhyCellId", "ssbPosition"]

# limpia string de DISTNAME
nrcell_df_mergelte["DISTNAME"] = nrcell_df_mergelte["DISTNAME"].astype(str).str.strip().str.upper()
nrdsslte_df["DISTNAME"] = nrdsslte_df["DISTNAME"].astype(str).str.strip().str.upper()

coincidencias = set(nrcell_df_mergelte["DISTNAME"]) & set(nrdsslte_df["DISTNAME"])
print(len(coincidencias))

nrcell_df_mergelte = nrcell_df_mergelte.merge(
    nrdsslte_df[["DISTNAME"] + columnas_a_insertar],
    on = "DISTNAME",
    how = "left")

print("Shape nuevo:", nrcell_df_mergelte.shape)

# Vista de verificación (muestra solo unas filas)
pd.set_option("display.max_columns", None)  # opcional
try:
    display(nrcell_df_mergelte.head(5))
except NameError:
    # Por si no estás en notebook
    print(nrcell_df_mergelte.head(5).to_string(index=False))


Shape original: (2159, 24)
2159
Shape nuevo: (2159, 27)


,AT&T_Site_Name,Site ID,VERSION,DISTNAME,MOID,angle,name,actDl256Qam,administrativeState,availabilityStatus,cellBarred,cellName,freqBandIndicatorNR,lcrId,nrCellIdentity,nrCellType,operationalState,pMax,physCellId,arfcnSsbPbch,chBwDl,chBwUl,nrarfcnDl,nrarfcnUl,enbPlmn_mcc_mnc_mncLength,ltePhyCellId,ssbPosition
0,GUASAL0477,110705,NRBTSCL24R1_2322_120,PLMN-PLMN/MRBTS-110705/NRBTS-110705/NRCELL-11,106423692,2.0,NMRGUASAL0477_1_T,true,Unlocked,NaN,notBarred,NMRGUASAL0477_1_T,7,11,1813790731,4DL4UL,enabled,43.0,60,531850,20MHz,20MHz,532000,508000,334&50&3,60,pos1
1,GUASAL0477,110705,NRBTSCL24R1_2322_120,PLMN-PLMN/MRBTS-110705/NRBTS-110705/NRCELL-12,106423694,4.0,NMRGUASAL0477_2_T,true,Unlocked,NaN,notBarred,NMRGUASAL0477_2_T,7,12,1813790732,4DL4UL,enabled,43.0,61,531850,20MHz,20MHz,532000,508000,334&50&3,61,pos1
2,GUASAL0477,110705,NRBTSCL24R1_2322_120,PLMN-PLMN/MRBTS-110705/NRBTS-110705/NRCELL-13,106423695,5.0,NMRGUASAL0477_3_T,true,Unlocked,NaN,notBarred,NMRGUASAL0477_3_T,7,13,1813790733,4DL4UL,enabled,43.0,62,531850,20MHz,20MHz,532000,508000,334&50&3,62,pos1
3,GUACEL1699,110724,NRBTSCL24R1_2322_120,PLMN-PLMN/MRBTS-110724/NRBTS-110724/NRCELL-11,111941765,3.0,NMRGUACEL1699_1_T,true,Unlocked,NaN,notBarred,NMRGUACEL1699_1_T,7,11,1814102027,4DL4UL,enabled,43.0,113,531850,20MHz,20MHz,532000,508000,334&50&3,113,pos1
4,GUACEL1699,110724,NRBTSCL24R1_2322_120,PLMN-PLMN/MRBTS-110724/NRBTS-110724/NRCELL-12,111941763,3.0,NMRGUACEL1699_2_T,true,Unlocked,NaN,notBarred,NMRGUACEL1699_2_T,7,12,1814102028,4DL4UL,enabled,43.0,111,531850,20MHz,20MHz,532000,508000,334&50&3,111,pos1


In [7]:
# =====================================================================
# 4) Información de NRPLMNSET_NSA
# =====================================================================

nrcell_df_mergelmnset = nrcell_df_mergelte.copy()

print("Shape original:", nrcell_df_mergelmnset.shape)

# Remueve la ultima parte de DISTNAME en NRDSSLTE para que haga match con DISTNAME de NRCELL
nrplmnset_df['DISTNAME'] = nrplmnset_df['DISTNAME'].str.rsplit('/', n=1).str[0]

columnas_a_insertar = ["configuredEpsTac", "nrPlmnDNList"]

# limpia string de DISTNAME
nrcell_df_mergelmnset["DISTNAME"] = nrcell_df_mergelmnset["DISTNAME"].astype(str).str.strip().str.upper()
nrplmnset_df["DISTNAME"] = nrplmnset_df["DISTNAME"].astype(str).str.strip().str.upper()

coincidencias = set(nrcell_df_mergelmnset["DISTNAME"]) & set(nrplmnset_df["DISTNAME"])
print(len(coincidencias))

nrcell_df_mergelmnset = nrcell_df_mergelmnset.merge(
    nrplmnset_df[["DISTNAME"] + columnas_a_insertar],
    on = "DISTNAME",
    how = "left")

print("Shape nuevo:", nrcell_df_mergelmnset.shape)

# Vista de verificación (muestra solo unas filas)
pd.set_option("display.max_columns", None)  # opcional
try:
    display(nrcell_df_mergelmnset.head(5))
except NameError:
    # Por si no estás en notebook
    print(nrcell_df_mergelmnset.head(5).to_string(index=False))



Shape original: (2159, 27)
2159
Shape nuevo: (2159, 29)


,AT&T_Site_Name,Site ID,VERSION,DISTNAME,MOID,angle,name,actDl256Qam,administrativeState,availabilityStatus,cellBarred,cellName,freqBandIndicatorNR,lcrId,nrCellIdentity,nrCellType,operationalState,pMax,physCellId,arfcnSsbPbch,chBwDl,chBwUl,nrarfcnDl,nrarfcnUl,enbPlmn_mcc_mnc_mncLength,ltePhyCellId,ssbPosition,configuredEpsTac,nrPlmnDNList
0,GUASAL0477,110705,NRBTSCL24R1_2322_120,PLMN-PLMN/MRBTS-110705/NRBTS-110705/NRCELL-11,106423692,2.0,NMRGUASAL0477_1_T,true,Unlocked,NaN,notBarred,NMRGUASAL0477_1_T,7,11,1813790731,4DL4UL,enabled,43.0,60,531850,20MHz,20MHz,532000,508000,334&50&3,60,pos1,6170,MRBTS-110705/NRBTS-110705/NRPLMN-1;MRBTS-11070...
1,GUASAL0477,110705,NRBTSCL24R1_2322_120,PLMN-PLMN/MRBTS-110705/NRBTS-110705/NRCELL-12,106423694,4.0,NMRGUASAL0477_2_T,true,Unlocked,NaN,notBarred,NMRGUASAL0477_2_T,7,12,1813790732,4DL4UL,enabled,43.0,61,531850,20MHz,20MHz,532000,508000,334&50&3,61,pos1,6170,MRBTS-110705/NRBTS-110705/NRPLMN-1;MRBTS-11070...
2,GUASAL0477,110705,NRBTSCL24R1_2322_120,PLMN-PLMN/MRBTS-110705/NRBTS-110705/NRCELL-13,106423695,5.0,NMRGUASAL0477_3_T,true,Unlocked,NaN,notBarred,NMRGUASAL0477_3_T,7,13,1813790733,4DL4UL,enabled,43.0,62,531850,20MHz,20MHz,532000,508000,334&50&3,62,pos1,6170,MRBTS-110705/NRBTS-110705/NRPLMN-1;MRBTS-11070...
3,GUACEL1699,110724,NRBTSCL24R1_2322_120,PLMN-PLMN/MRBTS-110724/NRBTS-110724/NRCELL-11,111941765,3.0,NMRGUACEL1699_1_T,true,Unlocked,NaN,notBarred,NMRGUACEL1699_1_T,7,11,1814102027,4DL4UL,enabled,43.0,113,531850,20MHz,20MHz,532000,508000,334&50&3,113,pos1,6115,MRBTS-110724/NRBTS-110724/NRPLMN-1;MRBTS-11072...
4,GUACEL1699,110724,NRBTSCL24R1_2322_120,PLMN-PLMN/MRBTS-110724/NRBTS-110724/NRCELL-12,111941763,3.0,NMRGUACEL1699_2_T,true,Unlocked,NaN,notBarred,NMRGUACEL1699_2_T,7,12,1814102028,4DL4UL,enabled,43.0,111,531850,20MHz,20MHz,532000,508000,334&50&3,111,pos1,6115,MRBTS-110724/NRBTS-110724/NRPLMN-1;MRBTS-11072...


In [8]:
# =====================================================================
# 5) LAT/LON desde All_Nokia_5G_{YYYYMM} (mes anterior)
# =====================================================================
nrcell_df_mergelatlon = nrcell_df_mergelmnset.copy()

today = date.today()
prev_year  = today.year if today.month > 1 else today.year - 1
prev_month = today.month - 1 or 12
yyyymm = f"{prev_year}{prev_month:02d}"
print(yyyymm)

# an_path = BASE_DIR / f"All_Nokia_5G_{yyyymm}.xlsx"
an_path = BASE_DIR / f"All_Nokia_5G_20260121.xlsx"
an_df = pd.read_excel(an_path, usecols=["AT&T_Site_Name", "LAT", "LON"])
an_df["AT&T_Site_Name"] = an_df["AT&T_Site_Name"].astype(str).str.strip()
an_df = an_df.drop_duplicates(subset=["AT&T_Site_Name"], keep="first")
display(an_df.head(5))

nrcell_df_mergelatlon["AT&T_Site_Name"] = nrcell_df_mergelatlon["AT&T_Site_Name"].astype(str).str.strip()

merged_df = nrcell_df_mergelatlon.merge(
    an_df,
    on="AT&T_Site_Name",
    how="left",
    suffixes=("", "_an")
)

"""
merged = nrcell_df_mod.merge(
    an_df[["AT&T_Site_Name", "LAT", "LON"]],
    on = "AT&T_Site_Name",
    how="left")
for col in ["LAT", "LON"]:
    m = _is_blank(merged[col]) if col in merged.columns else pd.Series(True, index=merged.index)
    merged.loc[m, col] = merged.loc[m, col + "_an"]
    if col + "_an" in merged:
        merged.drop(columns=[col + "_an"], inplace=True)

faltan = (
    _is_blank(merged["LAT"]) |
    _is_blank(merged["LON"])
)

if not faltan.any():
    nrcell_df_mod = merged
    print("All_Nokia cubrió 100% (LAT/LON).")
    display(df_out.loc[:, ["AT&T_Site_Name","LAT","LON"]].head(5))
else:
    print(f"Quedan {int(faltan.sum())} filas con faltantes. Se aplica fallback EPT…") """

print("Shape nuevo:", merged_df.shape)

# Vista de verificación (muestra solo unas filas)
pd.set_option("display.max_columns", None)  # opcional
try:
    display(merged_df.head(5))
except NameError:
    # Por si no estás en notebook
    print(merged_df.head(5).to_string(index=False))


202601


,AT&T_Site_Name,LAT,LON
0,GUASAL0477,20.560011,-101.168181
3,GUACEL1699,20.511615,-100.795872
6,GUALEO0421,21.116713,-101.595124
9,GUACEL0140,20.547800,-100.822800
12,GUALEO1839,21.134910,-101.716986


Shape nuevo: (2159, 31)


,AT&T_Site_Name,Site ID,VERSION,DISTNAME,MOID,angle,name,actDl256Qam,administrativeState,availabilityStatus,cellBarred,cellName,freqBandIndicatorNR,lcrId,nrCellIdentity,nrCellType,operationalState,pMax,physCellId,arfcnSsbPbch,chBwDl,chBwUl,nrarfcnDl,nrarfcnUl,enbPlmn_mcc_mnc_mncLength,ltePhyCellId,ssbPosition,configuredEpsTac,nrPlmnDNList,LAT,LON
0,GUASAL0477,110705,NRBTSCL24R1_2322_120,PLMN-PLMN/MRBTS-110705/NRBTS-110705/NRCELL-11,106423692,2.0,NMRGUASAL0477_1_T,true,Unlocked,NaN,notBarred,NMRGUASAL0477_1_T,7,11,1813790731,4DL4UL,enabled,43.0,60,531850,20MHz,20MHz,532000,508000,334&50&3,60,pos1,6170,MRBTS-110705/NRBTS-110705/NRPLMN-1;MRBTS-11070...,20.560011,-101.168181
1,GUASAL0477,110705,NRBTSCL24R1_2322_120,PLMN-PLMN/MRBTS-110705/NRBTS-110705/NRCELL-12,106423694,4.0,NMRGUASAL0477_2_T,true,Unlocked,NaN,notBarred,NMRGUASAL0477_2_T,7,12,1813790732,4DL4UL,enabled,43.0,61,531850,20MHz,20MHz,532000,508000,334&50&3,61,pos1,6170,MRBTS-110705/NRBTS-110705/NRPLMN-1;MRBTS-11070...,20.560011,-101.168181
2,GUASAL0477,110705,NRBTSCL24R1_2322_120,PLMN-PLMN/MRBTS-110705/NRBTS-110705/NRCELL-13,106423695,5.0,NMRGUASAL0477_3_T,true,Unlocked,NaN,notBarred,NMRGUASAL0477_3_T,7,13,1813790733,4DL4UL,enabled,43.0,62,531850,20MHz,20MHz,532000,508000,334&50&3,62,pos1,6170,MRBTS-110705/NRBTS-110705/NRPLMN-1;MRBTS-11070...,20.560011,-101.168181
3,GUACEL1699,110724,NRBTSCL24R1_2322_120,PLMN-PLMN/MRBTS-110724/NRBTS-110724/NRCELL-11,111941765,3.0,NMRGUACEL1699_1_T,true,Unlocked,NaN,notBarred,NMRGUACEL1699_1_T,7,11,1814102027,4DL4UL,enabled,43.0,113,531850,20MHz,20MHz,532000,508000,334&50&3,113,pos1,6115,MRBTS-110724/NRBTS-110724/NRPLMN-1;MRBTS-11072...,20.511615,-100.795872
4,GUACEL1699,110724,NRBTSCL24R1_2322_120,PLMN-PLMN/MRBTS-110724/NRBTS-110724/NRCELL-12,111941763,3.0,NMRGUACEL1699_2_T,true,Unlocked,NaN,notBarred,NMRGUACEL1699_2_T,7,12,1814102028,4DL4UL,enabled,43.0,111,531850,20MHz,20MHz,532000,508000,334&50&3,111,pos1,6115,MRBTS-110724/NRBTS-110724/NRPLMN-1;MRBTS-11072...,20.511615,-100.795872


In [9]:
# =====================================================================
# 6) LAT/LON desde EPT (mas reciente)
# =====================================================================

if merged_df[["LAT", "LON"]].isna().any(axis=1).any():
    # Ejecuta el proceso si hay al menos un NaN en LAT o LON

    nan_count = merged_df[["LAT", "LON"]].isna().sum().sum()
    print("Se necesita buscar LAT/LON restantes en EPT")

    ruta_ept = BASE_DIR

    # Prefijo del archivo
    prefijo_ept = "EPT_ATT_UMTS_LTE_"

    # Busca archivo que empiece con el prefijo
    archivo = glob.glob(os.path.join(ruta_ept, f"{prefijo_ept}*.xlsx"))

    # Verifica si se encontró archivo
    if archivo:
        archivo_encontrado = archivo[0]
        nombre_archivo = os.path.basename(archivo_encontrado)

        # Lista de hojas a leer
        hojas_fijas = [
            "EPT_3G_LTE_OUTDOOR",
            "PLAN_OUTDOOR",
            "EPT_3G_LTE_INDOOR",
            "PLAN_INDOOR",
            "Eventos_Especiales"
        ]

        # Detecta automáticamente las hojas que contienen "Nokia" (para este vendor en particular)
        todas_las_hojas = pd.ExcelFile(archivo_encontrado, engine="openpyxl").sheet_names
        hojas_vendor = [h for h in todas_las_hojas if "nokia" in h.lower()]
        # Combina ambas listas (sin duplicar)
        hojas = list(dict.fromkeys(hojas_fijas + hojas_vendor))
        print(hojas)

        # Lee todas las hojas y agrega el nombre de la hoja en columna
        dfs = [
            pd.read_excel(archivo_encontrado, sheet_name=hoja, usecols=["AT&T_Site_Name", "Latitud", "Longitud"], engine="openpyxl")
            .assign(Hoja=hoja, Origen=nombre_archivo)
            for hoja in hojas
        ]

        # Concatena todo en un solo DataFrame
        df_EPT_inicial = pd.concat(dfs, ignore_index=True).drop_duplicates(subset=["AT&T_Site_Name"])

        # Unir con df principal (solo para los NaN)
        merged_df = merged_df.merge(df_EPT_inicial, on="AT&T_Site_Name", how="left", suffixes=("", "_extra"))
        merged_df["LAT"] = merged_df["LAT"].fillna(merged_df["Latitud"])
        merged_df["LON"] = merged_df["LON"].fillna(merged_df["Longitud"])
        merged_df = merged_df.drop(columns=["Latitud", "Longitud"])

        if merged_df[["LAT", "LON"]].isna().any(axis=1).any():
            nan_count = merged_df[["LAT", "LON"]].isna().sum().sum()
            print(nan_count, " LAT/LON no encontrados. Se producirá archivo excel con estos faltantes.")

        # Vista de verificación (muestra solo unas filas)
        pd.set_option("display.max_columns", None)  # opcional
        try:
            display(merged_df.head(5))
        except NameError:
            # Por si no estás en notebook
            print(merged_df.head(5).to_string(index=False))
    else:
        print("⚠️ No se encontró archivo EPT")
else:
    print("LAT y LON encontrados en su totalidad en archivo anterior. No se necesita EPT.")

pd.set_option("display.max_columns", None)  # opcional
try:
    display(merged_df.head(5))
except NameError:
    # Por si no estás en notebook
    print(merged_df.head(5).to_string(index=False))



Se necesita buscar LAT/LON restantes en EPT
['EPT_3G_LTE_OUTDOOR', 'PLAN_OUTDOOR', 'EPT_3G_LTE_INDOOR', 'PLAN_INDOOR', 'Eventos_Especiales', 'Overlay Nokia', 'R&R Nokia', 'Nokia_DSS']
6  LAT/LON no encontrados. Se producirá archivo excel con estos faltantes.


C:\Users\SCaracoza\AppData\Local\Temp\ipykernel_28776\823448871.py:48: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_EPT_inicial = pd.concat(dfs, ignore_index=True).drop_duplicates(subset=["AT&T_Site_Name"])


,AT&T_Site_Name,Site ID,VERSION,DISTNAME,MOID,angle,name,actDl256Qam,administrativeState,availabilityStatus,cellBarred,cellName,freqBandIndicatorNR,lcrId,nrCellIdentity,nrCellType,operationalState,pMax,physCellId,arfcnSsbPbch,chBwDl,chBwUl,nrarfcnDl,nrarfcnUl,enbPlmn_mcc_mnc_mncLength,ltePhyCellId,ssbPosition,configuredEpsTac,nrPlmnDNList,LAT,LON,Hoja,Origen
0,GUASAL0477,110705,NRBTSCL24R1_2322_120,PLMN-PLMN/MRBTS-110705/NRBTS-110705/NRCELL-11,106423692,2.0,NMRGUASAL0477_1_T,true,Unlocked,NaN,notBarred,NMRGUASAL0477_1_T,7,11,1813790731,4DL4UL,enabled,43.0,60,531850,20MHz,20MHz,532000,508000,334&50&3,60,pos1,6170,MRBTS-110705/NRBTS-110705/NRPLMN-1;MRBTS-11070...,20.560011,-101.168181,EPT_3G_LTE_OUTDOOR,EPT_ATT_UMTS_LTE_2026-02-13.xlsx
1,GUASAL0477,110705,NRBTSCL24R1_2322_120,PLMN-PLMN/MRBTS-110705/NRBTS-110705/NRCELL-12,106423694,4.0,NMRGUASAL0477_2_T,true,Unlocked,NaN,notBarred,NMRGUASAL0477_2_T,7,12,1813790732,4DL4UL,enabled,43.0,61,531850,20MHz,20MHz,532000,508000,334&50&3,61,pos1,6170,MRBTS-110705/NRBTS-110705/NRPLMN-1;MRBTS-11070...,20.560011,-101.168181,EPT_3G_LTE_OUTDOOR,EPT_ATT_UMTS_LTE_2026-02-13.xlsx
2,GUASAL0477,110705,NRBTSCL24R1_2322_120,PLMN-PLMN/MRBTS-110705/NRBTS-110705/NRCELL-13,106423695,5.0,NMRGUASAL0477_3_T,true,Unlocked,NaN,notBarred,NMRGUASAL0477_3_T,7,13,1813790733,4DL4UL,enabled,43.0,62,531850,20MHz,20MHz,532000,508000,334&50&3,62,pos1,6170,MRBTS-110705/NRBTS-110705/NRPLMN-1;MRBTS-11070...,20.560011,-101.168181,EPT_3G_LTE_OUTDOOR,EPT_ATT_UMTS_LTE_2026-02-13.xlsx
3,GUACEL1699,110724,NRBTSCL24R1_2322_120,PLMN-PLMN/MRBTS-110724/NRBTS-110724/NRCELL-11,111941765,3.0,NMRGUACEL1699_1_T,true,Unlocked,NaN,notBarred,NMRGUACEL1699_1_T,7,11,1814102027,4DL4UL,enabled,43.0,113,531850,20MHz,20MHz,532000,508000,334&50&3,113,pos1,6115,MRBTS-110724/NRBTS-110724/NRPLMN-1;MRBTS-11072...,20.511615,-100.795872,EPT_3G_LTE_OUTDOOR,EPT_ATT_UMTS_LTE_2026-02-13.xlsx
4,GUACEL1699,110724,NRBTSCL24R1_2322_120,PLMN-PLMN/MRBTS-110724/NRBTS-110724/NRCELL-12,111941763,3.0,NMRGUACEL1699_2_T,true,Unlocked,NaN,notBarred,NMRGUACEL1699_2_T,7,12,1814102028,4DL4UL,enabled,43.0,111,531850,20MHz,20MHz,532000,508000,334&50&3,111,pos1,6115,MRBTS-110724/NRBTS-110724/NRPLMN-1;MRBTS-11072...,20.511615,-100.795872,EPT_3G_LTE_OUTDOOR,EPT_ATT_UMTS_LTE_2026-02-13.xlsx


,AT&T_Site_Name,Site ID,VERSION,DISTNAME,MOID,angle,name,actDl256Qam,administrativeState,availabilityStatus,cellBarred,cellName,freqBandIndicatorNR,lcrId,nrCellIdentity,nrCellType,operationalState,pMax,physCellId,arfcnSsbPbch,chBwDl,chBwUl,nrarfcnDl,nrarfcnUl,enbPlmn_mcc_mnc_mncLength,ltePhyCellId,ssbPosition,configuredEpsTac,nrPlmnDNList,LAT,LON,Hoja,Origen
0,GUASAL0477,110705,NRBTSCL24R1_2322_120,PLMN-PLMN/MRBTS-110705/NRBTS-110705/NRCELL-11,106423692,2.0,NMRGUASAL0477_1_T,true,Unlocked,NaN,notBarred,NMRGUASAL0477_1_T,7,11,1813790731,4DL4UL,enabled,43.0,60,531850,20MHz,20MHz,532000,508000,334&50&3,60,pos1,6170,MRBTS-110705/NRBTS-110705/NRPLMN-1;MRBTS-11070...,20.560011,-101.168181,EPT_3G_LTE_OUTDOOR,EPT_ATT_UMTS_LTE_2026-02-13.xlsx
1,GUASAL0477,110705,NRBTSCL24R1_2322_120,PLMN-PLMN/MRBTS-110705/NRBTS-110705/NRCELL-12,106423694,4.0,NMRGUASAL0477_2_T,true,Unlocked,NaN,notBarred,NMRGUASAL0477_2_T,7,12,1813790732,4DL4UL,enabled,43.0,61,531850,20MHz,20MHz,532000,508000,334&50&3,61,pos1,6170,MRBTS-110705/NRBTS-110705/NRPLMN-1;MRBTS-11070...,20.560011,-101.168181,EPT_3G_LTE_OUTDOOR,EPT_ATT_UMTS_LTE_2026-02-13.xlsx
2,GUASAL0477,110705,NRBTSCL24R1_2322_120,PLMN-PLMN/MRBTS-110705/NRBTS-110705/NRCELL-13,106423695,5.0,NMRGUASAL0477_3_T,true,Unlocked,NaN,notBarred,NMRGUASAL0477_3_T,7,13,1813790733,4DL4UL,enabled,43.0,62,531850,20MHz,20MHz,532000,508000,334&50&3,62,pos1,6170,MRBTS-110705/NRBTS-110705/NRPLMN-1;MRBTS-11070...,20.560011,-101.168181,EPT_3G_LTE_OUTDOOR,EPT_ATT_UMTS_LTE_2026-02-13.xlsx
3,GUACEL1699,110724,NRBTSCL24R1_2322_120,PLMN-PLMN/MRBTS-110724/NRBTS-110724/NRCELL-11,111941765,3.0,NMRGUACEL1699_1_T,true,Unlocked,NaN,notBarred,NMRGUACEL1699_1_T,7,11,1814102027,4DL4UL,enabled,43.0,113,531850,20MHz,20MHz,532000,508000,334&50&3,113,pos1,6115,MRBTS-110724/NRBTS-110724/NRPLMN-1;MRBTS-11072...,20.511615,-100.795872,EPT_3G_LTE_OUTDOOR,EPT_ATT_UMTS_LTE_2026-02-13.xlsx
4,GUACEL1699,110724,NRBTSCL24R1_2322_120,PLMN-PLMN/MRBTS-110724/NRBTS-110724/NRCELL-12,111941763,3.0,NMRGUACEL1699_2_T,true,Unlocked,NaN,notBarred,NMRGUACEL1699_2_T,7,12,1814102028,4DL4UL,enabled,43.0,111,531850,20MHz,20MHz,532000,508000,334&50&3,111,pos1,6115,MRBTS-110724/NRBTS-110724/NRPLMN-1;MRBTS-11072...,20.511615,-100.795872,EPT_3G_LTE_OUTDOOR,EPT_ATT_UMTS_LTE_2026-02-13.xlsx


In [11]:
# =====================================================================
# 7) Archivo final en excel
# =====================================================================
# Eliminar columnas auxiliares si existen
merged_df = merged_df.drop(columns=[c for c in ["Hoja", "Origen"] if c in merged_df.columns])

# === 0) Config y fecha actual ===

today   = date.today()
yyyymm  = f"{today.year}{today.month:02d}{today.day:02d}"

final_excel = BASE_DIR / f"All_Nokia_5G_{yyyymm}.xlsx"
tmp_excel   = BASE_DIR / f"~tmp_All_Nokia_5G_{yyyymm}.xlsx"

# Usa tu DataFrame final en memoria
df_out = merged_df.copy()  # o df_sorted si ya lo traes ordenado

pd.set_option("display.max_columns", None)  # opcional
try:
    display(df_out.head(5))
except NameError:
    # Por si no estás en notebook
    print(df_out.head(5).to_string(index=False))

# === 2) Guardar sin formato, sin 'nan' ===
df_out.to_excel(final_excel, index=False, na_rep="")

# 2) Reabrir el MISMO archivo y aplicar formato
wb = load_workbook(final_excel)
ws = wb.active

ws.freeze_panes = "A2"
for col_idx, header in enumerate(HEADERS, start=1):
    cell = ws.cell(row=1, column=col_idx)
    cell.value = header
    cell.font = Font(name="Aptos Narrow", size=11)
    cell.alignment = Alignment(textRotation=90, horizontal="center", vertical="center", wrap_text=True)
    cell.border = Border()

# Agregar filtro automático en el header
ws.auto_filter.ref = ws.dimensions  # aplica el filtro a todo el rango con datos

wb.save(final_excel)
wb.close()

print(f"✅ Archivo final sin formato guardado → {final_excel}")

,AT&T_Site_Name,Site ID,VERSION,DISTNAME,MOID,angle,name,actDl256Qam,administrativeState,availabilityStatus,cellBarred,cellName,freqBandIndicatorNR,lcrId,nrCellIdentity,nrCellType,operationalState,pMax,physCellId,arfcnSsbPbch,chBwDl,chBwUl,nrarfcnDl,nrarfcnUl,enbPlmn_mcc_mnc_mncLength,ltePhyCellId,ssbPosition,configuredEpsTac,nrPlmnDNList,LAT,LON
0,GUASAL0477,110705,NRBTSCL24R1_2322_120,PLMN-PLMN/MRBTS-110705/NRBTS-110705/NRCELL-11,106423692,2.0,NMRGUASAL0477_1_T,true,Unlocked,NaN,notBarred,NMRGUASAL0477_1_T,7,11,1813790731,4DL4UL,enabled,43.0,60,531850,20MHz,20MHz,532000,508000,334&50&3,60,pos1,6170,MRBTS-110705/NRBTS-110705/NRPLMN-1;MRBTS-11070...,20.560011,-101.168181
1,GUASAL0477,110705,NRBTSCL24R1_2322_120,PLMN-PLMN/MRBTS-110705/NRBTS-110705/NRCELL-12,106423694,4.0,NMRGUASAL0477_2_T,true,Unlocked,NaN,notBarred,NMRGUASAL0477_2_T,7,12,1813790732,4DL4UL,enabled,43.0,61,531850,20MHz,20MHz,532000,508000,334&50&3,61,pos1,6170,MRBTS-110705/NRBTS-110705/NRPLMN-1;MRBTS-11070...,20.560011,-101.168181
2,GUASAL0477,110705,NRBTSCL24R1_2322_120,PLMN-PLMN/MRBTS-110705/NRBTS-110705/NRCELL-13,106423695,5.0,NMRGUASAL0477_3_T,true,Unlocked,NaN,notBarred,NMRGUASAL0477_3_T,7,13,1813790733,4DL4UL,enabled,43.0,62,531850,20MHz,20MHz,532000,508000,334&50&3,62,pos1,6170,MRBTS-110705/NRBTS-110705/NRPLMN-1;MRBTS-11070...,20.560011,-101.168181
3,GUACEL1699,110724,NRBTSCL24R1_2322_120,PLMN-PLMN/MRBTS-110724/NRBTS-110724/NRCELL-11,111941765,3.0,NMRGUACEL1699_1_T,true,Unlocked,NaN,notBarred,NMRGUACEL1699_1_T,7,11,1814102027,4DL4UL,enabled,43.0,113,531850,20MHz,20MHz,532000,508000,334&50&3,113,pos1,6115,MRBTS-110724/NRBTS-110724/NRPLMN-1;MRBTS-11072...,20.511615,-100.795872
4,GUACEL1699,110724,NRBTSCL24R1_2322_120,PLMN-PLMN/MRBTS-110724/NRBTS-110724/NRCELL-12,111941763,3.0,NMRGUACEL1699_2_T,true,Unlocked,NaN,notBarred,NMRGUACEL1699_2_T,7,12,1814102028,4DL4UL,enabled,43.0,111,531850,20MHz,20MHz,532000,508000,334&50&3,111,pos1,6115,MRBTS-110724/NRBTS-110724/NRPLMN-1;MRBTS-11072...,20.511615,-100.795872


✅ Archivo final sin formato guardado → C:\Users\SCaracoza\Documents\AT&T\LST Cell Ran\Nokia\XML_Output\Febrero\All_Nokia_5G_20260218.xlsx
